[View in Colaboratory](https://colab.research.google.com/github/mholtzscher/eda-notebooks/blob/master/tensorflow/cnn_mnist.ipynb)

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
tf.__version__

'1.6.0'

In [0]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu
    )
    
    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu
    )
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        'classes': tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        'probabilities': tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step()
        )
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset('mnist')
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [5]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir=None)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpqmn0_1dj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f923852d0b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
tensors_to_log = {'probabilities': 'softmax_tensor'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [0]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
#     hooks=[logging_hook]
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqmn0_1dj/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2001 into /tmp/tmpqmn0_1dj/model.ckpt.
INFO:tensorflow:loss = 0.7888314, step = 2001
INFO:tensorflow:global_step/sec: 69.2761
INFO:tensorflow:loss = 0.56066406, step = 2101 (1.448 sec)
INFO:tensorflow:global_step/sec: 76.7528
INFO:tensorflow:loss = 0.56740546, step = 2201 (1.303 sec)
INFO:tensorflow:global_step/sec: 76.884
INFO:tensorflow:loss = 0.6859985, step = 2301 (1.303 sec)
INFO:tensorflow:global_step/sec: 76.5834
INFO:tensorflow:loss = 0.4640859, step = 2401 (1.304 sec)
INFO:tensorflow:global_step/sec: 76.8199
INFO:tensorflow:loss = 0.5607012, step = 2501 (1.302 sec)
INFO:tensorflow:global_step/sec: 77.0265
INFO:tensorflow:loss = 0.59

INFO:tensorflow:global_step/sec: 76.9246
INFO:tensorflow:loss = 0.22411422, step = 4801 (1.298 sec)
INFO:tensorflow:global_step/sec: 77.338
INFO:tensorflow:loss = 0.35066834, step = 4901 (1.298 sec)
INFO:tensorflow:global_step/sec: 76.725
INFO:tensorflow:loss = 0.39756355, step = 5001 (1.302 sec)
INFO:tensorflow:global_step/sec: 75.8282
INFO:tensorflow:loss = 0.18573934, step = 5101 (1.322 sec)
INFO:tensorflow:global_step/sec: 76.1094
INFO:tensorflow:loss = 0.26314217, step = 5201 (1.314 sec)
INFO:tensorflow:global_step/sec: 77.1872
INFO:tensorflow:loss = 0.42288363, step = 5301 (1.291 sec)
INFO:tensorflow:global_step/sec: 77.3203
INFO:tensorflow:loss = 0.35128027, step = 5401 (1.294 sec)
INFO:tensorflow:global_step/sec: 77.5893
INFO:tensorflow:loss = 0.19733846, step = 5501 (1.287 sec)
INFO:tensorflow:global_step/sec: 77.1239
INFO:tensorflow:loss = 0.34031516, step = 5601 (1.298 sec)
INFO:tensorflow:global_step/sec: 77.3053
INFO:tensorflow:loss = 0.3275563, step = 5701 (1.294 sec)
INF

INFO:tensorflow:global_step/sec: 76.7857
INFO:tensorflow:loss = 0.3210565, step = 7901 (1.302 sec)
INFO:tensorflow:global_step/sec: 77.0921
INFO:tensorflow:loss = 0.2507333, step = 8001 (1.301 sec)
INFO:tensorflow:global_step/sec: 76.6782
INFO:tensorflow:loss = 0.20936364, step = 8101 (1.301 sec)
INFO:tensorflow:global_step/sec: 76.7346
INFO:tensorflow:loss = 0.33556753, step = 8201 (1.304 sec)
INFO:tensorflow:global_step/sec: 77.0002
INFO:tensorflow:loss = 0.22796775, step = 8301 (1.295 sec)
INFO:tensorflow:global_step/sec: 76.7929
INFO:tensorflow:loss = 0.25018874, step = 8401 (1.304 sec)
INFO:tensorflow:global_step/sec: 77.1684
INFO:tensorflow:loss = 0.20341393, step = 8501 (1.299 sec)
INFO:tensorflow:global_step/sec: 77.6068
INFO:tensorflow:loss = 0.40396369, step = 8601 (1.287 sec)
INFO:tensorflow:global_step/sec: 77.6028
INFO:tensorflow:loss = 0.25515, step = 8701 (1.288 sec)
INFO:tensorflow:global_step/sec: 76.8295
INFO:tensorflow:loss = 0.2218352, step = 8801 (1.305 sec)
INFO:t

INFO:tensorflow:global_step/sec: 77.6521
INFO:tensorflow:loss = 0.24954721, step = 11001 (1.290 sec)
INFO:tensorflow:global_step/sec: 77.235
INFO:tensorflow:loss = 0.29244477, step = 11101 (1.292 sec)
INFO:tensorflow:global_step/sec: 76.065
INFO:tensorflow:loss = 0.42166552, step = 11201 (1.317 sec)
INFO:tensorflow:global_step/sec: 77.1674
INFO:tensorflow:loss = 0.15904798, step = 11301 (1.295 sec)
INFO:tensorflow:global_step/sec: 76.956
INFO:tensorflow:loss = 0.14561634, step = 11401 (1.297 sec)
INFO:tensorflow:global_step/sec: 77.0595
INFO:tensorflow:loss = 0.20039935, step = 11501 (1.301 sec)
INFO:tensorflow:global_step/sec: 77.1933
INFO:tensorflow:loss = 0.14350913, step = 11601 (1.292 sec)
INFO:tensorflow:global_step/sec: 76.8246
INFO:tensorflow:loss = 0.13648798, step = 11701 (1.305 sec)
INFO:tensorflow:global_step/sec: 77.0718
INFO:tensorflow:loss = 0.088518634, step = 11801 (1.296 sec)
INFO:tensorflow:global_step/sec: 77.3987
INFO:tensorflow:loss = 0.15757927, step = 11901 (1.2

INFO:tensorflow:global_step/sec: 76.3079
INFO:tensorflow:loss = 0.2256389, step = 14101 (1.308 sec)
INFO:tensorflow:global_step/sec: 77.1854
INFO:tensorflow:loss = 0.19792816, step = 14201 (1.295 sec)
INFO:tensorflow:global_step/sec: 76.9314
INFO:tensorflow:loss = 0.13035893, step = 14301 (1.301 sec)
INFO:tensorflow:global_step/sec: 76.8324
INFO:tensorflow:loss = 0.16875859, step = 14401 (1.302 sec)
INFO:tensorflow:global_step/sec: 76.9098
INFO:tensorflow:loss = 0.18169208, step = 14501 (1.301 sec)
INFO:tensorflow:global_step/sec: 77.376
INFO:tensorflow:loss = 0.1085682, step = 14601 (1.294 sec)
INFO:tensorflow:global_step/sec: 76.7808
INFO:tensorflow:loss = 0.08739165, step = 14701 (1.300 sec)
INFO:tensorflow:global_step/sec: 77.2506
INFO:tensorflow:loss = 0.20396154, step = 14801 (1.295 sec)
INFO:tensorflow:global_step/sec: 76.7499
INFO:tensorflow:loss = 0.12683915, step = 14901 (1.304 sec)
INFO:tensorflow:global_step/sec: 76.5205
INFO:tensorflow:loss = 0.102195434, step = 15001 (1.3

INFO:tensorflow:global_step/sec: 77.2725
INFO:tensorflow:loss = 0.112346, step = 17201 (1.292 sec)
INFO:tensorflow:global_step/sec: 76.9884
INFO:tensorflow:loss = 0.30684507, step = 17301 (1.300 sec)
INFO:tensorflow:global_step/sec: 76.3179
INFO:tensorflow:loss = 0.14121439, step = 17401 (1.310 sec)
INFO:tensorflow:global_step/sec: 76.8032
INFO:tensorflow:loss = 0.12267217, step = 17501 (1.302 sec)
INFO:tensorflow:global_step/sec: 76.6634
INFO:tensorflow:loss = 0.0990324, step = 17601 (1.304 sec)
INFO:tensorflow:global_step/sec: 76.9301
INFO:tensorflow:loss = 0.06542277, step = 17701 (1.303 sec)
INFO:tensorflow:global_step/sec: 76.8486
INFO:tensorflow:loss = 0.1136521, step = 17801 (1.299 sec)
INFO:tensorflow:global_step/sec: 76.8651
INFO:tensorflow:loss = 0.12179576, step = 17901 (1.302 sec)
INFO:tensorflow:global_step/sec: 76.8897
INFO:tensorflow:loss = 0.061522882, step = 18001 (1.300 sec)
INFO:tensorflow:global_step/sec: 76.8843
INFO:tensorflow:loss = 0.24638811, step = 18101 (1.30

INFO:tensorflow:global_step/sec: 76.842
INFO:tensorflow:loss = 0.1254339, step = 20301 (1.296 sec)
INFO:tensorflow:global_step/sec: 77.201
INFO:tensorflow:loss = 0.08835115, step = 20401 (1.296 sec)
INFO:tensorflow:global_step/sec: 76.7892
INFO:tensorflow:loss = 0.12479818, step = 20501 (1.307 sec)
INFO:tensorflow:global_step/sec: 76.7912
INFO:tensorflow:loss = 0.14179412, step = 20601 (1.299 sec)
INFO:tensorflow:global_step/sec: 76.1752
INFO:tensorflow:loss = 0.07233499, step = 20701 (1.310 sec)
INFO:tensorflow:global_step/sec: 76.6809
INFO:tensorflow:loss = 0.07276534, step = 20801 (1.307 sec)
INFO:tensorflow:global_step/sec: 75.9878
INFO:tensorflow:loss = 0.065406814, step = 20901 (1.314 sec)
INFO:tensorflow:global_step/sec: 77.2137
INFO:tensorflow:loss = 0.18037926, step = 21001 (1.294 sec)


In [8]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False
)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-17-18:12:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqmn0_1dj/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-17-18:12:35
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.8445, global_step = 2000, loss = 0.654013
{'accuracy': 0.8445, 'loss': 0.654013, 'global_step': 2000}
